# **Chapter 12: Binary Search Trees (BST)**

> *"A binary search tree is to search what a sorted array is to binary search—it imposes order on a hierarchical structure, enabling efficient dynamic set operations."* — Donald Knuth

---

## **12.1 Introduction to Binary Search Trees**

A **Binary Search Tree (BST)** is a binary tree that satisfies the **BST property**:

- For every node `x`:
  - All keys in the left subtree of `x` are **less than** `x.key`
  - All keys in the right subtree of `x` are **greater than** `x.key`

This property enables efficient search, insertion, and deletion operations, making BSTs fundamental for implementing dynamic sets and maps.

```
         50
       /    \
     30      70
    /  \    /  \
   20  40  60  80
```

### **12.1.1 Why BSTs Matter**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF BSTs                                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. DYNAMIC SET OPERATIONS: Insert, delete, search in O(h) time     │
│     where h is height. In balanced trees, h = O(log n).             │
│                                                                      │
│  2. ORDERED DATA: Inorder traversal yields sorted order, enabling   │
│     range queries, min/max, predecessor/successor efficiently.      │
│                                                                      │
│  3. FOUNDATION FOR ADVANCED TREES: AVL, Red-Black, B-Trees are      │
│     all variations of BSTs with balancing.                          │
│                                                                      │
│  4. UNDERLYING IMPLEMENTATION: Many language libraries use balanced │
│     BSTs for sorted maps and sets (e.g., C++ std::map, Java TreeMap).│
│                                                                      │
│  5. ALGORITHM DESIGN PATTERNS: BSTs appear in geometric algorithms, │
│     interval trees, order-statistic trees, etc.                     │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## **12.2 BST Node Structure and Basic Operations**

### **12.2.1 Node Definition**

```python
from typing import Optional, Any, List, Callable

class BSTNode:
    """Node for Binary Search Tree."""
    def __init__(self, key: Any, value: Any = None):
        self.key = key          # for comparison
        self.value = value      # associated data (optional)
        self.left: Optional['BSTNode'] = None
        self.right: Optional['BSTNode'] = None
        self.parent: Optional['BSTNode'] = None  # helpful for some operations

    def __repr__(self):
        return f"BSTNode({self.key})"
```

---

### **12.2.2 BST Class and Search**

Search follows the BST property recursively.

```python
class BinarySearchTree:
    def __init__(self):
        self.root: Optional[BSTNode] = None
        self.size = 0

    def search(self, key: Any) -> Optional[BSTNode]:
        """Return node with given key, or None if not found."""
        return self._search(self.root, key)

    def _search(self, node: Optional[BSTNode], key: Any) -> Optional[BSTNode]:
        if node is None or node.key == key:
            return node
        if key < node.key:
            return self._search(node.left, key)
        else:
            return self._search(node.right, key)
```

**Time complexity:** O(h), where h is tree height.

---

### **12.2.3 Insertion**

Insert a new key into the BST, maintaining BST property.

```python
def insert(self, key: Any, value: Any = None) -> None:
    """Insert key-value pair into BST."""
    new_node = BSTNode(key, value)
    if self.root is None:
        self.root = new_node
        self.size += 1
        return

    parent = None
    curr = self.root
    while curr:
        parent = curr
        if key < curr.key:
            curr = curr.left
        elif key > curr.key:
            curr = curr.right
        else:
            # Key already exists; handle by replacing value (or raise error)
            curr.value = value
            return

    # Attach new node
    if key < parent.key:
        parent.left = new_node
    else:
        parent.right = new_node
    new_node.parent = parent
    self.size += 1
```

---

### **12.2.4 Finding Minimum and Maximum**

```python
def minimum(self, node: Optional[BSTNode] = None) -> Optional[BSTNode]:
    """Return node with minimum key in subtree rooted at node."""
    if node is None:
        node = self.root
    if node is None:
        return None
    while node.left:
        node = node.left
    return node

def maximum(self, node: Optional[BSTNode] = None) -> Optional[BSTNode]:
    """Return node with maximum key in subtree rooted at node."""
    if node is None:
        node = self.root
    if node is None:
        return None
    while node.right:
        node = node.right
    return node
```

---

### **12.2.5 Successor and Predecessor**

- **Successor** of node `x` is the node with smallest key greater than `x.key`.
- **Predecessor** is the node with largest key smaller than `x.key`.

```python
def successor(self, node: BSTNode) -> Optional[BSTNode]:
    """Return inorder successor of node."""
    if node.right:
        return self.minimum(node.right)
    # Go up until we find a node that is the left child of its parent
    parent = node.parent
    while parent and parent.right == node:
        node = parent
        parent = parent.parent
    return parent

def predecessor(self, node: BSTNode) -> Optional[BSTNode]:
    """Return inorder predecessor of node."""
    if node.left:
        return self.maximum(node.left)
    parent = node.parent
    while parent and parent.left == node:
        node = parent
        parent = parent.parent
    return parent
```

---

### **12.2.6 Deletion**

Deletion is the most complex BST operation, with three cases:

1. **Node has no children** → simply remove it.
2. **Node has one child** → replace node with its child.
3. **Node has two children** → find successor (or predecessor), copy its key/value to node, then delete successor.

```python
def delete(self, key: Any) -> bool:
    """Delete node with given key. Return True if key existed."""
    node = self.search(key)
    if node is None:
        return False

    self._delete_node(node)
    self.size -= 1
    return True

def _delete_node(self, node: BSTNode) -> None:
    """Delete node from tree."""
    # Case 1: No children
    if node.left is None and node.right is None:
        self._replace_node(node, None)
    # Case 2: One child
    elif node.left is None:
        self._replace_node(node, node.right)
    elif node.right is None:
        self._replace_node(node, node.left)
    # Case 3: Two children
    else:
        # Find successor (or predecessor)
        successor = self.minimum(node.right)
        # Copy successor's data to node
        node.key, node.value = successor.key, successor.value
        # Delete successor recursively (will be case 1 or 2)
        self._delete_node(successor)

def _replace_node(self, node: BSTNode, new_node: Optional[BSTNode]) -> None:
    """Replace node with new_node (or None). Updates parent links."""
    if node.parent is None:
        self.root = new_node
    elif node == node.parent.left:
        node.parent.left = new_node
    else:
        node.parent.right = new_node
    if new_node:
        new_node.parent = node.parent
```

---

### **12.2.7 Inorder Traversal (Sorted Order)**

```python
def inorder(self) -> List[Any]:
    """Return list of keys in sorted order."""
    result = []
    self._inorder(self.root, result)
    return result

def _inorder(self, node: Optional[BSTNode], result: List[Any]) -> None:
    if node:
        self._inorder(node.left, result)
        result.append(node.key)
        self._inorder(node.right, result)
```

---

### **12.2.8 Example Usage**

```python
def demonstrate_bst():
    bst = BinarySearchTree()
    keys = [50, 30, 20, 40, 70, 60, 80]
    for k in keys:
        bst.insert(k)

    print("Inorder traversal:", bst.inorder())
    print("Search 40:", bst.search(40) is not None)
    print("Minimum:", bst.minimum().key)
    print("Maximum:", bst.maximum().key)

    node40 = bst.search(40)
    print("Successor of 40:", bst.successor(node40).key)
    print("Predecessor of 40:", bst.predecessor(node40).key)

    bst.delete(20)
    print("After deleting 20:", bst.inorder())
    bst.delete(30)
    print("After deleting 30:", bst.inorder())
    bst.delete(50)
    print("After deleting 50:", bst.inorder())

demonstrate_bst()
```

**Output:**
```
Inorder traversal: [20, 30, 40, 50, 60, 70, 80]
Search 40: True
Minimum: 20
Maximum: 80
Successor of 40: 50
Predecessor of 40: 30
After deleting 20: [30, 40, 50, 60, 70, 80]
After deleting 30: [40, 50, 60, 70, 80]
After deleting 50: [40, 60, 70, 80]
```

---

## **12.3 Analysis of BST Operations**

The time complexity of BST operations depends on the **height** of the tree.

- **Best case:** tree is balanced, height ≈ log₂ n → operations O(log n).
- **Worst case:** tree becomes a chain (like a linked list) when keys are inserted in sorted order → height = n → operations O(n).

```
Worst-case BST (degenerate):
1
 \
  2
   \
    3
     \
      4
```

- **Average case:** For random insertions, height ≈ O(log n) with constant factor.

---

## **12.4 Self-Balancing BSTs**

To guarantee O(log n) performance in the worst case, we use **self-balancing** BSTs that automatically maintain balance after insertions and deletions.

### **12.4.1 AVL Trees**

Named after Adelson-Velsky and Landis, AVL trees maintain the invariant that for every node, the heights of left and right subtrees differ by at most 1 (**balance factor** ∈ {-1, 0, 1}).

#### **AVL Node with Height**

```python
class AVLNode(BSTNode):
    def __init__(self, key, value=None):
        super().__init__(key, value)
        self.height = 1  # leaf height = 1
```

#### **Balance Factor**

```
balance_factor(node) = height(node.left) - height(node.right)
```

#### **Rotations**

When balance factor becomes ±2, we perform rotations to restore balance:

- **LL (Right Rotation)**: Occurs when left subtree is heavier and left child's left subtree is heavier.
- **RR (Left Rotation)**: Occurs when right subtree is heavier and right child's right subtree is heavier.
- **LR (Left-Right Rotation)**: Left subtree heavier, but left child's right subtree is heavy → first left rotate left child, then right rotate node.
- **RL (Right-Left Rotation)**: Right subtree heavier, but right child's left subtree is heavy → first right rotate right child, then left rotate node.

```python
def _right_rotate(self, y):
    """Right rotation around y (y is imbalanced node)."""
    x = y.left
    T2 = x.right

    # Perform rotation
    x.right = y
    y.left = T2

    # Update heights
    y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))
    x.height = 1 + max(self._get_height(x.left), self._get_height(x.right))

    return x  # new root

def _left_rotate(self, x):
    """Left rotation around x."""
    y = x.right
    T2 = y.left

    y.left = x
    x.right = T2

    x.height = 1 + max(self._get_height(x.left), self._get_height(x.right))
    y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))

    return y
```

#### **AVL Insertion**

Insert recursively, then update heights and balance.

```python
def insert(self, key, value=None):
    self.root = self._insert(self.root, key, value)

def _insert(self, node, key, value):
    # 1. Perform normal BST insertion
    if not node:
        return AVLNode(key, value)
    if key < node.key:
        node.left = self._insert(node.left, key, value)
    elif key > node.key:
        node.right = self._insert(node.right, key, value)
    else:
        # duplicate keys not allowed; update value
        node.value = value
        return node

    # 2. Update height
    node.height = 1 + max(self._get_height(node.left),
                          self._get_height(node.right))

    # 3. Get balance factor
    balance = self._get_balance(node)

    # 4. Balance the node if needed
    # Left Left Case
    if balance > 1 and key < node.left.key:
        return self._right_rotate(node)
    # Right Right Case
    if balance < -1 and key > node.right.key:
        return self._left_rotate(node)
    # Left Right Case
    if balance > 1 and key > node.left.key:
        node.left = self._left_rotate(node.left)
        return self._right_rotate(node)
    # Right Left Case
    if balance < -1 and key < node.right.key:
        node.right = self._right_rotate(node.right)
        return self._left_rotate(node)

    return node
```

**Deletion** follows similar pattern: perform BST deletion, then update heights and rebalance.

AVL trees provide stricter balancing than Red-Black trees, leading to faster lookups but slightly slower insertions/deletions due to more rotations.

---

### **12.4.2 Red-Black Trees**

Red-Black trees are another balanced BST variant, used in many libraries (C++ `std::map`, Java `TreeMap`). They maintain balance using color bits (red/black) and five invariants:

1. Every node is either red or black.
2. The root is always black.
3. Every leaf (NIL) is black.
4. If a node is red, both its children are black (no two consecutive reds).
5. For each node, all paths from the node to descendant leaves contain the same number of black nodes (black-height).

These invariants ensure that the longest path from root to leaf is at most twice the shortest path, so height ≤ 2 log₂(n+1).

#### **Red-Black Tree Insertion**

Insertion in a Red-Black tree:
1. Perform standard BST insertion, color new node **red**.
2. Fix violations:
   - If new node is root, color it black.
   - If parent is black, done.
   - If parent is red, we have a double-red violation. Fix by:
     - Recoloring and/or rotations, depending on uncle's color.
3. After fixing, ensure root is black.

#### **Red-Black Tree Deletion**

Deletion is more complex, with cases to restore black-height and red-black properties. The algorithm uses rotations and recoloring, and may involve a "double black" situation.

Due to space, we outline the concepts; full implementation is lengthy but available in standard references.

---

### **12.4.3 Splay Trees**

Splay trees are a self-adjusting BST where recently accessed elements move to the root via **splaying** operations. They provide amortized O(log n) time for all operations.

**Splaying** is a series of rotations that bring a node to the root, based on its relationship with its parent and grandparent:

- **Zig**: node is child of root → single rotation.
- **Zig-Zig**: node and parent are both left (or right) children → two rotations (same direction).
- **Zig-Zag**: node is left child of right child (or vice versa) → double rotation opposite direction.

```python
def splay(self, key):
    """Bring node with key to root if exists; else last accessed node."""
    # (Implementation not shown for brevity)
```

Splay trees are used in cache implementations and garbage collection due to their locality properties.

---

### **12.4.4 Treaps (Randomized BST)**

A **Treap** (Tree + Heap) combines a BST with a heap property. Each node has a key (BST order) and a randomly assigned priority (heap order: parent priority > children priorities).

- Insertion: insert as in BST, then rotate up while heap property violated.
- Deletion: rotate down until leaf, then remove.

Because priorities are random, the tree is balanced with high probability (expected height O(log n)).

```python
import random

class TreapNode:
    def __init__(self, key, value=None):
        self.key = key
        self.value = value
        self.priority = random.random()
        self.left = None
        self.right = None

def rotate_right(y):
    x = y.left
    T2 = x.right
    x.right = y
    y.left = T2
    return x

def rotate_left(x):
    y = x.right
    T2 = y.left
    y.left = x
    x.right = T2
    return y

def insert(root, key, value=None):
    if not root:
        return TreapNode(key, value)
    if key < root.key:
        root.left = insert(root.left, key, value)
        if root.left.priority > root.priority:
            root = rotate_right(root)
    else:
        root.right = insert(root.right, key, value)
        if root.right.priority > root.priority:
            root = rotate_left(root)
    return root
```

Treaps are simpler to implement than AVL or Red-Black trees and perform well in practice.

---

## **12.5 Comparison of Balanced BSTs**

```
┌────────────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ Property           │ AVL Tree     │ Red-Black    │ Splay Tree   │ Treap        │
├────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Worst-case height  │ 1.44 log n   │ 2 log n      │ O(n) but     │ O(n) with    │
│                    │              │              │ amortized    │ very low     │
│                    │              │              │ O(log n)     │ probability  │
├────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Lookup             │ O(log n)     │ O(log n)     │ amortized    │ O(log n)     │
│                    │              │              │ O(log n)     │ expected     │
├────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Insert/Delete      │ O(log n)     │ O(log n)     │ amortized    │ O(log n)     │
│                    │ many rotations│ few rotations│ O(log n)     │ expected     │
├────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Implementation     │ Moderate     │ Complex      │ Moderate     │ Simple       │
│ Complexity         │              │              │              │              │
├────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Extra storage      │ Height field │ Color bit    │ None         │ Priority     │
└────────────────────┴──────────────┴──────────────┴──────────────┴──────────────┘
```

---

## **12.6 Applications of BSTs**

```python
def bst_applications():
    """
    Real-world applications of Binary Search Trees.
    """
    print("\nApplications of BSTs")
    print("=" * 70)
    print("""
    1. Database Indexing
       - B-Trees (generalization of BSTs) used in relational databases
       - In-memory indexes often use balanced BSTs

    2. Language Libraries
       - C++ std::map, std::set (Red-Black trees)
       - Java TreeMap, TreeSet (Red-Black trees)
       - Python's dict is hash table, but sortedcontainers uses BST

    3. File Systems
       - Directory structures (hierarchical) can be traversed like BST

    4. Network Routing
       - IP routing tables (trie is a multi-way tree)

    5. Geometric Algorithms
       - Segment trees, interval trees (built on BSTs)

    6. Priority Queues
       - Treap can implement priority queue with additional operations

    7. Order Statistics
       - Augmented BSTs can support rank queries (OS trees)

    8. Compiler Design
       - Symbol tables (often implemented as hash tables or BSTs)
    """)

bst_applications()
```

---

## **12.7 Summary and Complexities**

```python
def bst_summary():
    print("\nBinary Search Tree Complexities")
    print("=" * 70)
    print("""
    ─────────────────────────────────────────────────────────────────────
    Operation               │ BST (worst) │ Balanced BST (worst)
    ─────────────────────────────────────────────────────────────────────
    Search                  │ O(n)        │ O(log n)
    Insert                  │ O(n)        │ O(log n)
    Delete                  │ O(n)        │ O(log n)
    Minimum/Maximum         │ O(n)        │ O(log n) or O(log n)*
    Successor/Predecessor   │ O(n)        │ O(log n)
    Inorder traversal       │ O(n)        │ O(n)

    * if tree is balanced; if we keep min pointer, O(1)

    Space: O(n) for all.
    """)

bst_summary()
```

---

## **12.8 Practice Problems**

### **Problem 1: Validate Binary Search Tree**
Given a binary tree, determine if it is a valid BST.

**Hint**: Use inorder traversal (should be sorted) or recursion with min/max bounds.

### **Problem 2: Lowest Common Ancestor in BST**
Given a BST and two nodes, find their LCA.

**Hint**: Use BST property: LCA is the first node where keys lie on different sides.

### **Problem 3: Kth Smallest Element in BST**
Find the kth smallest element in a BST.

**Hint**: Inorder traversal; or augment each node with size of subtree.

### **Problem 4: Convert Sorted Array to Balanced BST**
Given a sorted array, build a height-balanced BST.

**Hint**: Recursively pick middle as root.

### **Problem 5: Binary Search Tree Iterator**
Implement an iterator that returns elements in inorder.

**Hint**: Use stack to simulate recursion.

### **Problem 6: Serialize and Deserialize BST**
Design an algorithm to serialize and deserialize a BST (compact representation possible because of BST property).

**Hint**: Use preorder with bounds to reconstruct.

### **Problem 7: Two Sum in BST**
Given a BST and a target, find if there exist two elements that sum to target.

**Hint**: Inorder traversal + two pointers, or hash set.

---

## **12.9 Further Reading**

1. **"Introduction to Algorithms" (CLRS)** - Chapter 12 (Binary Search Trees), Chapter 13 (Red-Black Trees)
2. **"The Art of Computer Programming, Vol 3: Sorting and Searching"** by Donald Knuth - Section 6.2.2 (Binary Tree Searching)
3. **"Algorithms"** by Robert Sedgewick and Kevin Wayne - Chapter 3 (Searching)
4. **"Data Structures and Algorithm Analysis in C++"** by Mark Allen Weiss - Chapters 4 & 5
5. **Original Papers**:
   - Adelson-Velsky and Landis (1962) - AVL trees
   - Bayer (1972) - Red-Black trees (originally symmetric binary B-trees)
   - Sleator and Tarjan (1985) - Splay trees
   - Aragon and Seidel (1989) - Treaps

---

> **Coming in Chapter 13**: **Specialized Trees** - We'll explore B-Trees, Segment Trees, Fenwick Trees, Tries, and more, extending tree concepts to solve specific problems.

---

**End of Chapter 12**